<a href="https://colab.research.google.com/github/mbakersf/amazon-linformer/blob/main/amazon_training_title_kv_compressed_%3D_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## IMDB Dataset

In [1]:
# Setup
!pip install fairseq

# Download the IMDB dataset
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar zxvf aclImdb_v1.tar.gz

Streaming output truncated to the last 5000 lines.
aclImdb/train/unsup/44983_0.txt
aclImdb/train/unsup/44982_0.txt
aclImdb/train/unsup/44981_0.txt
aclImdb/train/unsup/44980_0.txt
aclImdb/train/unsup/44979_0.txt
aclImdb/train/unsup/44978_0.txt
aclImdb/train/unsup/44977_0.txt
aclImdb/train/unsup/44976_0.txt
aclImdb/train/unsup/44975_0.txt
aclImdb/train/unsup/44974_0.txt
aclImdb/train/unsup/44973_0.txt
aclImdb/train/unsup/44972_0.txt
aclImdb/train/unsup/44971_0.txt
aclImdb/train/unsup/44970_0.txt
aclImdb/train/unsup/44969_0.txt
aclImdb/train/unsup/44968_0.txt
aclImdb/train/unsup/44967_0.txt
aclImdb/train/unsup/44966_0.txt
aclImdb/train/unsup/44965_0.txt
aclImdb/train/unsup/44964_0.txt
aclImdb/train/unsup/44963_0.txt
aclImdb/train/unsup/44962_0.txt
aclImdb/train/unsup/44961_0.txt
aclImdb/train/unsup/44960_0.txt
aclImdb/train/unsup/44959_0.txt
aclImdb/train/unsup/44958_0.txt
aclImdb/train/unsup/44957_0.txt
aclImdb/train/unsup/44956_0.txt
aclImdb/train/unsup/44955_0.txt
aclImdb/train/unsup/4

In [2]:
# Format data
import os
import random
from glob import glob

def prepare_data(datadir):
    random.seed(0)
    for split in ['train', 'test']:
        samples = []
        for class_label in ['pos', 'neg']:
            fnames = glob(os.path.join(datadir, split, class_label) + '/*.txt')
            for fname in fnames:
                with open(fname, 'r') as fin:
                    line = fin.readline().strip()
                    samples.append((line, 1 if class_label == 'pos' else 0))
        random.shuffle(samples)
        out_fname = 'train' if split == 'train' else 'dev'
        with open(os.path.join(datadir, out_fname + '.input0'), 'w') as f1, \
             open(os.path.join(datadir, out_fname + '.label'), 'w') as f2:
            for sample in samples:
                f1.write(sample[0] + '\n')
                f2.write(str(sample[1]) + '\n')

prepare_data('aclImdb')

In [3]:
!ls

aclImdb  aclImdb_v1.tar.gz  sample_data


In [4]:
!git clone https://github.com/pytorch/fairseq
%cd fairseq

Cloning into 'fairseq'...
remote: Enumerating objects: 35184, done.
remote: Counting objects: 100% (105/105), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 35184 (delta 61), reused 72 (delta 47), pack-reused 35079
Receiving objects: 100% (35184/35184), 25.22 MiB | 22.30 MiB/s, done.
Resolving deltas: 100% (25548/25548), done.
/content/fairseq


In [5]:
!ls

CODE_OF_CONDUCT.md  fairseq	   LICENSE	   RELEASE.md	     setup.py
CONTRIBUTING.md     fairseq_cli    MANIFEST.in	   release_utils.py  tests
docs		    hubconf.py	   pyproject.toml  scripts	     train.py
examples	    hydra_plugins  README.md	   setup.cfg


In [6]:
# Download the BPE encoder and vocabulary
!wget -N 'https://dl.fbaipublicfiles.com/fairseq/gpt2_bpe/encoder.json'
!wget -N 'https://dl.fbaipublicfiles.com/fairseq/gpt2_bpe/vocab.bpe'

# BPE encoding of the data
!python -m examples.roberta.multiprocessing_bpe_encoder \
    --encoder-json encoder.json \
    --vocab-bpe vocab.bpe \
    --inputs "../aclImdb/train.input0" "../aclImdb/dev.input0" \
    --outputs "../aclImdb/train.input0.bpe" "../aclImdb/dev.input0.bpe" \
    --workers 60 \
    --keep-empty

--2024-05-06 01:28:51--  https://dl.fbaipublicfiles.com/fairseq/gpt2_bpe/encoder.json
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.226.210.111, 13.226.210.25, 13.226.210.15, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.226.210.111|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1042301 (1018K) [text/plain]
Saving to: ‘encoder.json’

encoder.json        100%[===================>]   1018K  --.-KB/s    in 0.03s   

2024-05-06 01:28:51 (30.1 MB/s) - ‘encoder.json’ saved [1042301/1042301]

--2024-05-06 01:28:51--  https://dl.fbaipublicfiles.com/fairseq/gpt2_bpe/vocab.bpe
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.226.210.111, 13.226.210.25, 13.226.210.15, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.226.210.111|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 456318 (446K) [text/plain]
Saving to: ‘vocab.bpe’

vocab.bpe           100%[===================>

In [7]:
# Download the dictionary for fairseq
!wget -N 'https://dl.fbaipublicfiles.com/fairseq/gpt2_bpe/dict.txt'

# Preprocess the data for fairseq
!fairseq-preprocess \
    --only-source \
    --trainpref "../aclImdb/train.input0.bpe" \
    --validpref "../aclImdb/dev.input0.bpe" \
    --destdir "../IMDB-bin/input0" \
    --srcdict dict.txt \
    --workers 60

!fairseq-preprocess \
    --only-source \
    --trainpref "../aclImdb/train.label" \
    --validpref "../aclImdb/dev.label" \
    --destdir "../IMDB-bin/label" \
    --workers 60


--2024-05-06 01:33:57--  https://dl.fbaipublicfiles.com/fairseq/gpt2_bpe/dict.txt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.226.210.78, 13.226.210.25, 13.226.210.111, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.226.210.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 603290 (589K) [text/plain]
Saving to: ‘dict.txt’

dict.txt            100%[===================>] 589.15K  --.-KB/s    in 0.04s   

2024-05-06 01:33:57 (16.1 MB/s) - ‘dict.txt’ saved [603290/603290]

2024-05-06 01:33:59.571935: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-06 01:33:59.571990: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-06 01:33:59.573456: E externa

In [ ]:
!pip install tensorboardX

!fairseq-train "/content/IMDB-bin/" \
    --user-dir /content/fairseq/examples/linformer/linformer_src \
    --max-positions 512 \
    --batch-size 16 \
    --max-tokens 4400 \
    --task sentence_prediction \
    --reset-optimizer --reset-dataloader --reset-meters \
    --required-batch-size-multiple 1 \
    --init-token 0 --separator-token 2 \
    --arch linformer_roberta_base \
    --criterion sentence_prediction \
    --classification-head-name 'imdb_head' \
    --num-classes 2 \
    --dropout 0.1 --attention-dropout 0.1 \
    --weight-decay 0.1 --optimizer adam --adam-betas "(0.9, 0.98)" --adam-eps 1e-06 \
    --clip-norm 0.0 \
    --lr-scheduler polynomial_decay --lr 1e-05 --total-num-update 7812 --warmup-updates 469 \
    --fp16 --fp16-init-scale 4 --threshold-loss-scale 1 --fp16-scale-window 128 \
    --max-epoch 10 \
    --best-checkpoint-metric accuracy --maximize-best-checkpoint-metric \
    --shorten-method "truncate" \
    --find-unused-parameters \
    --update-freq 4 \
    --shared-kv-compressed = 1

2024-05-06 00:40:13.850806: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-06 00:40:13.850849: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-06 00:40:13.852211: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-06 00:40:13.859367: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Traceback (most recent call last):
  File "/u

## Twitter Dataset

In [ ]:
# Importing the dataset from Kaggle

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'sentiment140:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2477%2F4140%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240503%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240503T155156Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D287bc94575b702a6892fe02b7bd28c326a770f21bf9b3800966a2d7b086fdff55f5a34ec7f9085607ee9b47f7377889ebb681c276883014dae0098dc44c979cef2a1da717476acc16ba0c738273585d2cca0721c12a45af861cd7deb08724952a3f6ca8ba45ed6b55aa0c22fd0054cde92f216e6cf6388c84700e4d22af8a4955a7cd524e6a1e9956d8c8b981be1773b2f187f7b969c87de1c0fca659e5e68570477c3eb52066d453f36e6229e50514d6d21c1d460acdf4d22a2763469dfa07801404cfad4383ff87b396cedf172160af365557cebe9b40ec9a1838af60e49b5a8eb8ada61146332bb53e6e0db21087781df57d2613fa4a816bf965739a546dd'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')

In [ ]:
import pandas as pd
import tensorflow
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
import nltk
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords
import spacy
import re
import sklearn
from sklearn.model_selection import train_test_split
import tqdm
nltk.download('stopwords')
STOPWORDS = set(stopwords.words('english'))
from gensim.models import Word2Vec
from tensorflow.keras import Sequential
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.pipeline import make_pipeline
from tensorflow.keras import datasets,models,layers
from tensorflow.keras.layers import Conv1D, Concatenate, GlobalMaxPooling1D, GlobalAveragePooling1D, Dense, Embedding, Input,BatchNormalization
from tensorflow.keras.models import Model

In [ ]:
data = pd.read_csv('/kaggle/input/sentiment140/training.1600000.processed.noemoticon.csv',encoding = 'latin',header=None)
data = data[[5, 0]]
data.columns=['tweet', 'sentiment']
print(data.head())
data['sentiment'] = data['sentiment'].replace(4,1)
data = data.sample(frac = 1)
print(data.head())

In [ ]:
print(len(data))
data = data.head(160000)
print(len(data))

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd

# Split the data into training and validation sets
train_df, dev_df = train_test_split(data, test_size=0.1, random_state=42)

train_df['tweet'].to_csv('train.input0', index=False, header=False)
train_df['sentiment'].to_csv('train.label', index=False, header=False)
dev_df['tweet'].to_csv('dev.input0', index=False, header=False)
dev_df['sentiment'].to_csv('dev.label', index=False, header=False)

# Check number of lines in each file
print("Training texts:", len(open('train.input0').readlines()))
print("Training labels:", len(open('train.label').readlines()))
print("Validation texts:", len(open('dev.input0').readlines()))
print("Validation labels:", len(open('dev.label').readlines()))


In [ ]:
# # BPE encoding of the data
!python -m examples.roberta.multiprocessing_bpe_encoder \
    --encoder-json encoder.json \
    --vocab-bpe vocab.bpe \
    --inputs "train.input0" "dev.input0" \
    --outputs "train.input0.bpe" "dev.input0.bpe" \
    --workers 60 \
    --keep-empty

In [ ]:
!!wc -l train.input0
!wc -l dev.input0
!wc -l train.input0.bpe
!wc -l dev.input0.bpe

In [ ]:
!pip install tensorboardX

In [ ]:
# Preprocess the data for fairseq
!fairseq-preprocess \
    --only-source \
    --trainpref "train.input0.bpe" \
    --validpref "dev.input0.bpe" \
    --destdir "Tweet-bin/input0" \
    --srcdict dict.txt \
    --workers 60

!fairseq-preprocess \
    --only-source \
    --trainpref "train.label" \
    --validpref "dev.label" \
    --destdir "Tweet-bin/label" \
    --workers 60

In [ ]:
# Re-doing this process with a subset of the dataset
!rm -rf Tweet-bin/input0 Tweet-bin/label
!head -n 16000 train.label > train.small.label

!fairseq-preprocess \
    --only-source \
    --trainpref "train.input0.bpe" \
    --validpref "dev.input0.bpe" \
    --destdir "Tweet-bin/input0" \
    --srcdict dict.txt \
    --workers 60

!fairseq-preprocess \
    --only-source \
    --trainpref "train.small.label" \
    --validpref "dev.label" \
    --destdir "Tweet-bin/label" \
    --workers 60

In [ ]:
# Training with Linformer using the Twitter data
!fairseq-train "Tweet-bin/" \
    --user-dir /content/fairseq/examples/linformer/linformer_src \
    --max-positions 512 \
    --batch-size 16 \
    --max-tokens 4400 \
    --task sentence_prediction \
    --reset-optimizer --reset-dataloader --reset-meters \
    --required-batch-size-multiple 1 \
    --init-token 0 --separator-token 2 \
    --arch linformer_roberta_base \
    --criterion sentence_prediction \
    --classification-head-name 'sentiment_head' \
    --num-classes 2 \
    --dropout 0.1 --attention-dropout 0.1 \
    --weight-decay 0.1 --optimizer adam --adam-betas "(0.9, 0.98)" --adam-eps 1e-06 \
    --clip-norm 0.0 \
    --lr-scheduler polynomial_decay --lr 1e-05 --total-num-update 7812 --warmup-updates 469 \
    --fp16 --fp16-init-scale 4 --threshold-loss-scale 1 --fp16-scale-window 128 \
    --max-epoch 10 \
    --best-checkpoint-metric accuracy --maximize-best-checkpoint-metric \
    --shorten-method "truncate" \
    --find-unused-parameters \
    --update-freq 4

In [ ]:
# Load model and make predictions
from fairseq.models.roberta import RobertaModel
from fairseq.models import MODEL_REGISTRY

print(list(MODEL_REGISTRY.keys()))

import os
print(os.path.exists('checkpoints/checkpoint_best.pt'))  # Should return True

roberta = RobertaModel.from_pretrained(
    'checkpoints',
    checkpoint_file='checkpoint_best.pt',
    data_name_or_path='Tweet-bin'
)
roberta.eval()  # disable dropout

# Example prediction
tokens = roberta.encode('Best movie this year')
pred = roberta.predict('sentiment_head', tokens).argmax().item()
print('Prediction:', pred)  # Output: 1 (positive)

## Amazon Dataset


In [8]:
# Step 1: Install the Hugging Face `datasets` library
!pip install datasets

# Import the necessary library
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("amazon_polarity")

# Accessing data
train_data = dataset["train"]
test_data = dataset["test"]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 15.6 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/3600000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/400000 [00:00<?, ? examples/s]

In [9]:
print(dataset)
print(train_data)

DatasetDict({
    train: Dataset({
        features: ['label', 'title', 'content'],
        num_rows: 3600000
    })
    test: Dataset({
        features: ['label', 'title', 'content'],
        num_rows: 400000
    })
})
Dataset({
    features: ['label', 'title', 'content'],
    num_rows: 3600000
})


In [10]:
# Print some sample data
for example in train_data.shuffle(seed=42).select(range(5)):
    print(f"Label: {example['label']}, Review Title: {example['title']}, Review Content: {example['content']}")

Label: 0, Review Title: Anyone who likes this better than the Pekinpah is a moron., Review Content: All the pretty people in this film. Even the Rudy character played by Michael Madsen. This is adapted from a Jim Thompson novel for cryin' out loud! These are supposed to be marginal characters, not fashion models. Though McQueen and McGraw were attractive (but check out McQueen's crummy prison haircut) they were believable in the role. Baldwin and Bassinger seem like movie stars trying to act like hard cases. Action wise, the robbery scene in the Pekinpah version was about 100 times more exciting and suspenseful than anything in this re-make.
Label: 0, Review Title: Author seems mentally unstable, Review Content: I know that Tom Robbins has a loyal following and I started the book with high expectations. However, I did not enjoy this book as it was too much work to follow his confused logic. I think that he was under the influence during most of time that he wrote.
Label: 1, Review Titl

In [11]:
import pandas as pd

# Convert to pandas DataFrame
df_train = pd.DataFrame(train_data)
df_test = pd.DataFrame(test_data)

# Assuming binary classification where 'label' 0 and 1 are used
df_train['label'] = df_train['label'].replace({4: 1})
df_test['label'] = df_test['label'].replace({4: 1})

# Shuffle the DataFrame
df_train = df_train.sample(frac=1, random_state=42)
df_test = df_test.sample(frac=1, random_state=42)

df_train = df_train.head(25000)
df_test = df_test.head(25000)

print(df_train.head())

         label                                   title  \
2079998      0                          Expensive Junk   
1443106      0                          Toast too dark   
3463669      1   Excellent imagery...dumbed down story   
2914699      0  Are we pretending everyone is married?   
1603231      0                     Not worth your time   

                                                   content  
2079998  This product consists of a piece of thin flexi...  
1443106  Even on the lowest setting, the toast is too d...  
3463669  I enjoyed this disc. The video is stunning. I ...  
2914699  The authors pretend that parents neither die n...  
1603231  Might as well just use a knife, this product h...  


In [12]:
!ls

CODE_OF_CONDUCT.md  encoder.json  hubconf.py	 pyproject.toml    scripts    train.py
CONTRIBUTING.md     examples	  hydra_plugins  README.md	   setup.cfg  vocab.bpe
dict.txt	    fairseq	  LICENSE	 RELEASE.md	   setup.py
docs		    fairseq_cli   MANIFEST.in	 release_utils.py  tests


In [13]:
!ls

CODE_OF_CONDUCT.md  encoder.json  hubconf.py	 pyproject.toml    scripts    train.py
CONTRIBUTING.md     examples	  hydra_plugins  README.md	   setup.cfg  vocab.bpe
dict.txt	    fairseq	  LICENSE	 RELEASE.md	   setup.py
docs		    fairseq_cli   MANIFEST.in	 release_utils.py  tests


In [14]:
df_train['title'].to_csv('train.input0', index=False, header=False)
df_train['label'].to_csv('train.label', index=False, header=False)
df_test['title'].to_csv('dev.input0', index=False, header=False)
df_test['label'].to_csv('dev.label', index=False, header=False)

# Check number of lines in each file
print("Training texts:", len(open('train.input0').readlines()))
print("Training labels:", len(open('train.label').readlines()))
print("Validation texts:", len(open('dev.input0').readlines()))
print("Validation labels:", len(open('dev.label').readlines()))

Training texts: 25000
Training labels: 25000
Validation texts: 25000
Validation labels: 25000


In [15]:
!pip install tensorboardX

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 3.7 MB/s eta 0:00:00


In [16]:
# # BPE encoding of the data
!ls

!python -m examples.roberta.multiprocessing_bpe_encoder \
    --encoder-json encoder.json \
    --vocab-bpe vocab.bpe \
    --inputs "train.input0" \
    --outputs "train.input0.bpe" \
    --workers 60 \
    --keep-empty

!python -m examples.roberta.multiprocessing_bpe_encoder \
    --encoder-json encoder.json \
    --vocab-bpe vocab.bpe \
    --inputs "dev.input0" \
    --outputs "dev.input0.bpe" \
    --workers 60 \
    --keep-empty

# Preprocess the data for fairseq
!fairseq-preprocess \
    --only-source \
    --trainpref "train.input0.bpe" \
    --validpref "dev.input0.bpe" \
    --destdir "Amazon-bin/input0" \
    --srcdict dict.txt \
    --workers 60

!fairseq-preprocess \
    --only-source \
    --trainpref "train.label" \
    --validpref "dev.label" \
    --destdir "Amazon-bin/label" \
    --workers 60

CODE_OF_CONDUCT.md  docs	  hubconf.py	  README.md	    setup.py	  vocab.bpe
CONTRIBUTING.md     encoder.json  hydra_plugins   RELEASE.md	    tests
dev.input0	    examples	  LICENSE	  release_utils.py  train.input0
dev.label	    fairseq	  MANIFEST.in	  scripts	    train.label
dict.txt	    fairseq_cli   pyproject.toml  setup.cfg	    train.py
2024-05-06 01:49:02.605156: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-06 01:49:02.605201: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-06 01:49:02.606799: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-06 01:49:02.6142

In [17]:
!wget -N 'https://dl.fbaipublicfiles.com/fairseq/gpt2_bpe/dict.txt'

--2024-05-06 01:51:56--  https://dl.fbaipublicfiles.com/fairseq/gpt2_bpe/dict.txt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.226.210.78, 13.226.210.111, 13.226.210.25, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.226.210.78|:443... connected.
HTTP request sent, awaiting response... 304 Not Modified
File ‘dict.txt’ not modified on server. Omitting download.



In [ ]:
!ls

Amazon-bin	    dev.label	  fairseq	 MANIFEST.in	   scripts	 train.input0.bpe
CODE_OF_CONDUCT.md  dict.txt	  fairseq_cli	 pyproject.toml    setup.cfg	 train.label
CONTRIBUTING.md     docs	  hubconf.py	 README.md	   setup.py	 train.py
dev.input0	    encoder.json  hydra_plugins  RELEASE.md	   tests	 vocab.bpe
dev.input0.bpe	    examples	  LICENSE	 release_utils.py  train.input0


In [ ]:

!fairseq-train "Amazon-bin/" \
    --user-dir /content/fairseq/examples/linformer/linformer_src \
    --max-positions 512 \
    --batch-size 16 \
    --max-tokens 4400 \
    --task sentence_prediction \
    --reset-optimizer --reset-dataloader --reset-meters \
    --required-batch-size-multiple 1 \
    --init-token 0 --separator-token 2 \
    --arch linformer_roberta_base \
    --criterion sentence_prediction \
    --classification-head-name 'Amazon_head' \
    --num-classes 2 \
    --dropout 0.1 --attention-dropout 0.1 \
    --weight-decay 0.1 --optimizer adam --adam-betas "(0.9, 0.98)" --adam-eps 1e-06 \
    --clip-norm 0.0 \
    --lr-scheduler polynomial_decay --lr 1e-05 --total-num-update 7812 --warmup-updates 469 \
    --fp16 --fp16-init-scale 4 --threshold-loss-scale 1 --fp16-scale-window 128 \
    --max-epoch 10 \
    --best-checkpoint-metric accuracy --maximize-best-checkpoint-metric \
    --shorten-method "truncate" \
    --find-unused-parameters \
    --update-freq 4 \
    --shared-kv-compressed 1

2024-05-06 01:52:01.192843: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-06 01:52:01.192901: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-06 01:52:01.194241: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-06 01:52:01.201508: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-06 01:52:02.254020: W tensorflow/comp